## Binance top 200 coins by volume and momentum

In [25]:
coins = 50

### Obtengo la data usando la API de Binance

In [26]:
import requests
import pandas as pd
from datetime import datetime, timedelta

def get_top_volume_coins_list_api(count=200):
    # Obtener la lista de todos los símbolos de trading
    symbols_url = "https://api.binance.com/api/v3/exchangeInfo"
    symbols_response = requests.get(symbols_url)
    symbols_data = symbols_response.json()
    
    # Filtrar solo los pares con USDT
    usdt_symbols = [symbol['symbol'] for symbol in symbols_data['symbols'] if symbol['symbol'].endswith('USDT')]
    
    # Calcular la fecha de inicio (7 días atrás)
    start_time = int((datetime.now() - timedelta(days=7)).timestamp() * 1000)
    
    # Obtener el volumen para cada símbolo
    volumes = []
    for symbol in usdt_symbols:
        klines_url = f"https://api.binance.com/api/v3/klines?symbol={symbol}&interval=1d&startTime={start_time}"
        klines_response = requests.get(klines_url)
        klines_data = klines_response.json()
        
        # Calcular el volumen promedio
        total_volume = sum(float(kline[7]) for kline in klines_data) #7 es quote asset volume
        avg_volume = total_volume / len(klines_data) if klines_data else 0
        volumes.append((symbol[:-4], avg_volume))  # Removemos 'USDT' del símbolo
    
    # Ordenar por volumen descendente y tomar los primeros 'count'
    top_volumes = sorted(volumes, key=lambda x: x[1], reverse=True)[:count]
    
    return top_volumes  # Ahora retornamos la lista completa de tuplas (moneda, volumen)

def get_top_volume_coins_df_api(count=200):
    volumes = get_top_volume_coins_list_api(count)
    df = pd.DataFrame(volumes, columns=['coin', 'average volume last 7 days'])

    # Ordenar el DataFrame antes de formatear los números
    df = df.sort_values('average volume last 7 days', ascending=False)

   # Formatear los números después de ordenar
    df['average volume last 7 days'] = df['average volume last 7 days'].apply(lambda x: f'{x:.2f}')

    return df

In [27]:
api_top_coins_list = get_top_volume_coins_list_api(coins)
api_top_coins_list

[('BTC', 1223833607.2532868),
 ('ETH', 557900318.7265012),
 ('USDC', 446692116.0431857),
 ('SOL', 357365713.2217885),
 ('FDUSD', 335872544.8537571),
 ('DOGS', 124759616.5776907),
 ('PEPE', 105846273.1587772),
 ('BNB', 98568148.7959),
 ('WIF', 96244072.33311142),
 ('XRP', 91250005.38972856),
 ('1000SATS', 67810331.56640866),
 ('FLOKI', 64545448.92993728),
 ('FET', 64361232.05855715),
 ('TRX', 52191474.42414285),
 ('SUN', 45621955.71900429),
 ('PEOPLE', 43537780.50514257),
 ('DOGE', 40822771.50204142),
 ('TON', 39218739.35889143),
 ('ORDI', 38482654.881157145),
 ('SUI', 34322877.57318429),
 ('NOT', 33267451.99042257),
 ('NEAR', 31244365.74482857),
 ('AAVE', 31190465.512815714),
 ('RUNE', 29822765.49627143),
 ('AVAX', 29408118.07961428),
 ('MATIC', 29080194.460422855),
 ('FTM', 25551678.673299994),
 ('BOME', 25355884.57218371),
 ('ZRO', 24527230.22503714),
 ('RDNT', 24377314.8629),
 ('LTC', 23313694.54174),
 ('ADA', 23289114.695764285),
 ('SHIB', 20970910.528678123),
 ('CRV', 20536240.425

In [28]:
api_top_coins_df = get_top_volume_coins_df_api(coins)
api_top_coins_df

,coin,average volume last 7 days
0,BTC,1224110895.43
1,ETH,558107139.22
2,USDC,446783915.82
3,SOL,357439793.71
4,FDUSD,335953898.32
5,DOGS,124781556.67
6,PEPE,105891159.34
7,BNB,98587841.64
8,WIF,96266190.44
9,XRP,91264628.83


### Obtengo la data usando el wraper de Python de Binance

In [29]:
from binance.client import Client
import pandas as pd
from datetime import datetime, timedelta

# Inicializar el cliente de Binance
# Nota: Para el uso de la API pública, no necesitamos proporcionar claves API
client = Client()

def get_top_volume_coins_list_wraper(count=200):
    # Obtener la información de todos los símbolos de trading
    exchange_info = client.get_exchange_info()
    
    # Filtrar solo los pares con USDT
    usdt_symbols = [symbol['symbol'] for symbol in exchange_info['symbols'] if symbol['symbol'].endswith('USDT')]
    
    # Calcular la fecha de inicio (7 días atrás)
    start_time = int((datetime.now() - timedelta(days=7)).timestamp() * 1000)
    
    # Obtener el volumen para cada símbolo
    volumes = []
    for symbol in usdt_symbols:
        klines = client.get_klines(symbol=symbol, interval=Client.KLINE_INTERVAL_1DAY, startTime=start_time)
        
        # Calcular el volumen promedio
        total_volume = sum(float(kline[7]) for kline in klines) #7 es quote asset volume
        avg_volume = total_volume / len(klines) if klines else 0
        volumes.append((symbol[:-4], avg_volume))  # Removemos 'USDT' del símbolo
    
    # Ordenar por volumen descendente y tomar los primeros 'count'
    top_volumes = sorted(volumes, key=lambda x: x[1], reverse=True)[:count]
    
    return top_volumes  # Ahora retornamos la lista completa de tuplas (moneda, volumen)

def get_top_volume_coins_df_wraper(count=200):
    volumes = get_top_volume_coins_list_wraper(count)
    df = pd.DataFrame(volumes, columns=['coin', 'average volume last 7 days'])

    # Ordenar el DataFrame antes de formatear los números
    df = df.sort_values('average volume last 7 days', ascending=False)

    # Formatear los números después de ordenar
    df['average volume last 7 days'] = df['average volume last 7 days'].apply(lambda x: f'{x:.2f}')
    

    return df





In [30]:
#wraper_top_coins_list = get_top_volume_coins_list_wraper(coins)
#wraper_top_coins_list

In [31]:
#wraper_top_coins_df = get_top_volume_coins_df_wraper(coins)
#wraper_top_coins_df

### Conclusion
Mejor uso el API directamente, ya que evito que CS50 tenga que instalar un paquete adicional de python. 

Las variables que pasan a la siguiente etapa son: 


api_top_coins_df 

api_top_coins_list

Trabajare con la lista pero primero tengo que eliminar los valores de volumen, que hago a continuacion

In [37]:
top_coins = [item[0] for item in api_top_coins_list]
top_coins

['BTC',
 'ETH',
 'USDC',
 'SOL',
 'FDUSD',
 'DOGS',
 'PEPE',
 'BNB',
 'WIF',
 'XRP',
 '1000SATS',
 'FLOKI',
 'FET',
 'TRX',
 'SUN',
 'PEOPLE',
 'DOGE',
 'TON',
 'ORDI',
 'SUI',
 'NOT',
 'NEAR',
 'AAVE',
 'RUNE',
 'AVAX',
 'MATIC',
 'FTM',
 'BOME',
 'ZRO',
 'RDNT',
 'LTC',
 'ADA',
 'SHIB',
 'CRV',
 'EUR',
 'ARB',
 'ENA',
 'APT',
 'RARE',
 'WLD',
 'PENDLE',
 'RENDER',
 'IO',
 'HARD',
 'SEI',
 'EURI',
 'BONK',
 'TAO',
 'LINK',
 'VIDT']

## Obtener datos para cada criptomoeda

In [38]:
import pandas as pd
from binance.client import Client
from datetime import datetime, timedelta
import os
import sqlite3

client = Client()

def get_crypto_data(crypto_list, lookback, reference_low):
    # Convertir la fecha de referencia a timestamp
    reference_date = datetime.strptime(reference_low, "%d-%m-%Y")
    start_date = reference_date - timedelta(days=lookback)
    start_timestamp = int(start_date.timestamp() * 1000)

    dataframes = []

    # Crear la carpeta 'data' si no existe
    if not os.path.exists('data'):
        os.makedirs('data')

    for crypto in crypto_list:
        # Obtener los datos de Binance
        klines = client.get_historical_klines(
            symbol=f"{crypto}USDT",
            interval=Client.KLINE_INTERVAL_1DAY,
            start_str=start_timestamp
        )

        # Crear el DataFrame
        df = pd.DataFrame(klines, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'])
        
        # Convertir el timestamp a datetime
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
        
        # Establecer el timestamp como índice
        df.set_index('timestamp', inplace=True)
        
        # Convertir las columnas necesarias a float
        for col in ['open', 'high', 'low', 'close', 'volume']:
            df[col] = df[col].astype(float)
        
        # Guardar el DataFrame como CSV
        filename = f"data/{crypto}_DF_{lookback}.csv"
        df.to_csv(filename)
        
        dataframes.append(df)
        print(f"Datos de {crypto} guardados en {filename}")

    return dataframes

def create_db(crypto_list):
    # Crear la base de datos
    conn = sqlite3.connect('crypto_db.db')
    cursor = conn.cursor()

    # Crear una tabla para cada criptomoneda
    for crypto in crypto_list:
        cursor.execute(f'''
        CREATE TABLE IF NOT EXISTS {crypto} (
            date TEXT PRIMARY KEY,
            open REAL,
            high REAL,
            low REAL,
            close REAL,
            volume REAL
        )
        ''')

    conn.commit()
    conn.close()
    print("Base de datos creada: crypto_db.db")

# Ejemplo de uso:
# crypto_list = ['BTC', 'ETH', 'ADA']
# dataframes = get_crypto_data(crypto_list, 30, "01-06-2023")
# create_db(crypto_list)

In [39]:
dataframes = get_crypto_data(top_coins, 600, "05-07-2024")

Datos de BTC guardados en data/BTC_DF_600.csv
Datos de ETH guardados en data/ETH_DF_600.csv
Datos de USDC guardados en data/USDC_DF_600.csv
Datos de SOL guardados en data/SOL_DF_600.csv
Datos de FDUSD guardados en data/FDUSD_DF_600.csv
Datos de DOGS guardados en data/DOGS_DF_600.csv
Datos de PEPE guardados en data/PEPE_DF_600.csv
Datos de BNB guardados en data/BNB_DF_600.csv
Datos de WIF guardados en data/WIF_DF_600.csv
Datos de XRP guardados en data/XRP_DF_600.csv
Datos de 1000SATS guardados en data/1000SATS_DF_600.csv
Datos de FLOKI guardados en data/FLOKI_DF_600.csv
Datos de FET guardados en data/FET_DF_600.csv
Datos de TRX guardados en data/TRX_DF_600.csv
Datos de SUN guardados en data/SUN_DF_600.csv
Datos de PEOPLE guardados en data/PEOPLE_DF_600.csv
Datos de DOGE guardados en data/DOGE_DF_600.csv
Datos de TON guardados en data/TON_DF_600.csv
Datos de ORDI guardados en data/ORDI_DF_600.csv
Datos de SUI guardados en data/SUI_DF_600.csv
Datos de NOT guardados en data/NOT_DF_600.csv


In [41]:
dataframes[0]

,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,ignore
timestamp,,,,,,,,,,,
2022-11-14,16331.78,17190.00,15815.21,16619.46,380210.77750,1668470399999,6273920325.33682730,9215128,189502.41248000,3127495594.21076420,0
2022-11-15,16617.72,17134.69,16527.72,16900.57,282461.84391,1668556799999,4758545355.78683170,7200265,141015.70658000,2375750102.41831840,0
2022-11-16,16900.57,17015.92,16378.61,16662.76,261493.40809,1668643199999,4362538338.56532390,6297404,129644.72244000,2162757428.72151800,0
2022-11-17,16661.61,16751.00,16410.74,16692.56,228038.97873,1668729599999,3785204632.01422130,5352083,113347.39771000,1881536908.06648980,0
2022-11-18,16692.56,17011.00,16546.04,16700.45,214224.18184,1668815999999,3588467560.37312090,5114799,106366.35569000,1781845353.09400520,0
...,...,...,...,...,...,...,...,...,...,...,...
2024-08-31,59123.99,59462.38,58744.00,58973.99,8798.40900,1725148799999,519849184.86904460,895944,4195.93079000,247955494.35915920,0
2024-09-01,58974.00,59076.59,57201.00,57301.86,20705.15741,1725235199999,1202176847.39105850,2153180,9651.50531000,560452387.89854740,0
2024-09-02,57301.77,59425.69,57128.00,59132.13,22895.01461,1725321599999,1333092369.44656860,1966119,11295.25452000,657496577.63048030,0
